In [12]:
import traitlets
import ipywidgets
import ipywidgets.widgets as widgets
from IPython.display import display
from traitlets.config.configurable import SingletonConfigurable
import numpy as np
import enum
import subprocess
import cv2
import sys
sys.path.append('..')
from opencv_course.video import Camera

size_slider = ipywidgets.FloatSlider(min=1.0, max=5.0, step=0.2, value=1.0, description='Size')

def bgr8_to_jpeg(value, quality=10):
    global size_slider
    size = size_slider.value
    (w,h) = value.shape[0:2]
    image = value[int(w*(1-1/size)/2):int(w*(1+1/size)/2), int(h*(1-1/size)/2):int(h*(1+1/size)/2)]
    
    return bytes(cv2.imencode('.jpg', image)[1])

display(size_slider)

FloatSlider(value=1.0, description='Size', max=5.0, min=1.0, step=0.2)

## 测试摄像头支持的分辨率

常用的分辨率可以通过在wiki查询，https://en.wikipedia.org/wiki/List_of_common_resolutions<br>
如果找到支持的分辨率，则能正常显示图像

In [ ]:
for i in [(2304,1536),
            (1920,1080),
            (1280,720),
            (1024,576),
            (960,540),
            (848,480),
            (800,600),
            (640,480),
            (640,360),
            (320,240),
            (160,120)]:
    print("Trying resulution {}".format(i))
    (w, h) = i
    try:
        camera = Camera(width=w, height=h, fps=2, is_usb=True)

        image = widgets.Image(format='jpeg', width=240, fps=24, height=180)  # this width and height doesn't necessarily have to match the camera

        traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

        display(image)
        
        camera.unobserve_all()
        camera.stop()
    except:
        print("Error {}".format(i))
        pass
    

In [ ]:
w=160
h=120
for i in range(2, 30, 1):
    print("Trying fps {}".format(i))
    try:
        camera = Camera(width=w, height=h, fps=i, is_usb=True)

        image = widgets.Image(format='jpeg', width=240, height=180)  # this width and height doesn't necessarily have to match the camera

        traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

        display(image)
        
        camera.unobserve_all()
        camera.stop()
    except:
        print("Error {}".format(i))
        pass

In [ ]:
w = 848
h = 480
fps =24

camera = Camera(width=w, height=h, fps=24, is_usb=True)

image = widgets.Image(format='jpeg', width=240, fps=24, height=180)  # this width and height doesn't necessarily have to match the camera

traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

In [ ]:
camera.unobserve_all()
camera.stop()

## 通过命令行修改UVC摄像头参数

首先我门要检查一下摄像头有什么参数, 命令是

v4l2-ctl -d <video device> -l

In [4]:
!v4l2-ctl -d 1 -l

                     brightness 0x00980900 (int)    : min=0 max=255 step=1 default=128 value=191
                       contrast 0x00980901 (int)    : min=0 max=255 step=1 default=128 value=157
                     saturation 0x00980902 (int)    : min=0 max=255 step=1 default=128 value=136
 white_balance_temperature_auto 0x0098090c (bool)   : default=1 value=1
                           gain 0x00980913 (int)    : min=0 max=255 step=1 default=0 value=203
           power_line_frequency 0x00980918 (menu)   : min=0 max=2 default=2 value=2
      white_balance_temperature 0x0098091a (int)    : min=2000 max=7500 step=1 default=4000 value=4000 flags=inactive
                      sharpness 0x0098091b (int)    : min=0 max=255 step=1 default=128 value=128
         backlight_compensation 0x0098091c (int)    : min=0 max=1 step=1 default=0 value=1
                  exposure_auto 0x009a0901 (menu)   : min=0 max=3 default=3 value=3
              exposure_absolute 0x009a0902 (int)    : min=3 max=2047

我们可以使用subprocess.check_output()来获取当前数值

In [6]:
subprocess.check_output(["v4l2-ctl", "-d", "1", "-l" ])

b'                     brightness 0x00980900 (int)    : min=0 max=255 step=1 default=128 value=191\n                       contrast 0x00980901 (int)    : min=0 max=255 step=1 default=128 value=157\n                     saturation 0x00980902 (int)    : min=0 max=255 step=1 default=128 value=136\n white_balance_temperature_auto 0x0098090c (bool)   : default=1 value=1\n                           gain 0x00980913 (int)    : min=0 max=255 step=1 default=0 value=203\n           power_line_frequency 0x00980918 (menu)   : min=0 max=2 default=2 value=2\n      white_balance_temperature 0x0098091a (int)    : min=2000 max=7500 step=1 default=4000 value=4000 flags=inactive\n                      sharpness 0x0098091b (int)    : min=0 max=255 step=1 default=128 value=128\n         backlight_compensation 0x0098091c (int)    : min=0 max=1 step=1 default=0 value=1\n                  exposure_auto 0x009a0901 (menu)   : min=0 max=3 default=3 value=3\n              exposure_absolute 0x009a0902 (int)    : mi

In [10]:
from __future__ import print_function
from ipywidgets import interact
import subprocess

import ipywidgets as widgets

def f(brightness):
    subprocess.check_output(["v4l2-ctl", "-d", "1", "-c", "brightness={}".format(brightness)])
    return brightness

interact(f, brightness=widgets.IntSlider(min=0, max=255, step=1, value=228));
        

interactive(children=(IntSlider(value=228, description='brightness', max=255), Output()), _dom_classes=('widge…

In [13]:
w=320
h=240
fps=10
camera = Camera(width=w, height=h, fps=fps, is_usb=True)

image = widgets.Image(format='jpeg', width=240, height=180)  # this width and height doesn't necessarily have to match the camera

traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

w:2304.0, h:1536.0, fps:2.0, brightness:228.0, contrast:157.0, zoom:0.0
h, w: 240, 320


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [14]:
camera.unobserve_all()
camera.stop()